---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09


* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;


* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009


* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009


* Feb 2009; Sep 2009; Oct 2010


* 6/2008; 12/2009


* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [288]:
import pandas as pd

doc = []
with open('data\dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [289]:
# first, we need to know how many medical note we're dealing with.
len(df)

500

## - Instead of what we did above we can just create a Regex for each single scenario of date.

In most programming languages, whitespace `\s` is defined to mean spaces, tabs, carriage returns, newlines, and maybe a few other characters that behave similarly: 
- `-` don't belong to \s so you need to write it separately.

- Then, non-whitespace `\S` would be all the other characters. This would include all the alphabetical and numeric characters, punctuation`, belongs`, etc.

![variation_list](variation_list.png)

In [290]:
s1 = '(\d{1,2}[\/|\-]\d{1,2}[\/|\-|\ ]\d{2,4})'
s2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\-\s]\d{1,2}[,]{0,1}[\-\s]\d{2,4})'
s3 = '(\d{1,2}[\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\-|\s]\d{2,4})'
s4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\-|\s]\d{2,4}[\S]*[\s]\d{2,4})'
s5 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\-|\s]\d{2,4})'
s6 = '(\d{1,2}[\/|\-][1|2]\d{3})'
s7 = '([1|2]\d{3})'

full_regex = '(%s|%s|%s|%s|%s|%s|%s)' %(s1, s2, s3, s4, s5, s6,s7)

parsed_date = df.str.extract(full_regex)

# To make sure that there is no NAN in the first column`0`. 
print(parsed_date.info())
parsed_date.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       500 non-null    object
 1   1       125 non-null    object
 2   2       34 non-null     object
 3   3       69 non-null     object
 4   4       0 non-null      object
 5   5       115 non-null    object
 6   6       112 non-null    object
 7   7       45 non-null     object
dtypes: object(8)
memory usage: 31.4+ KB
None


,0,1,2,3,4,5,6,7
0,03/25/93,03/25/93,NaN,NaN,NaN,NaN,NaN,NaN
1,6/18/85,6/18/85,NaN,NaN,NaN,NaN,NaN,NaN
2,7/8/71,7/8/71,NaN,NaN,NaN,NaN,NaN,NaN
3,9/27/75,9/27/75,NaN,NaN,NaN,NaN,NaN,NaN
4,2/6/96,2/6/96,NaN,NaN,NaN,NaN,NaN,NaN
5,7/06/79,7/06/79,NaN,NaN,NaN,NaN,NaN,NaN
6,5/18/78,5/18/78,NaN,NaN,NaN,NaN,NaN,NaN
7,10/24/89,10/24/89,NaN,NaN,NaN,NaN,NaN,NaN
8,3/7/86,3/7/86,NaN,NaN,NaN,NaN,NaN,NaN
9,4/10/71,4/10/71,NaN,NaN,NaN,NaN,NaN,NaN


In [291]:
# there was a problem in the line num 125&313 which is that i didn't write a 
# regex to match it so i needed to go above and write scenario_4.

# check:
print(parsed_date.iloc[313,0])
parsed_date.iloc[125,0]

Decemeber 1978


'24 Jan 2001'

In [292]:
# this was a problem that was giving us `67 5/2000` that lead
# to remove the `\space` from the s1. 
parsed_date.iloc[392,0]

'5/2000'

In [293]:
# now the time to convert these dates to a timestamp form 
# using pd.to_datetime():

# pd.to_datetime(parsed_date)
# i got :AttributeError: 'int' object has no attribute 'lower' 
# which means that there are spelling mistakes that to_datetime()
# doesn't understand.

# cut the df to make a series from 1st column.
parsed_date = parsed_date.iloc[:,0]

# correct the spelling mistakes.
parsed_date = parsed_date.str.replace('Janaury', 'January')
parsed_date = parsed_date.str.replace('Decemeber', 'December')

# finally, convert to datetime form.
# That will apply all the rules were given above.
parsed_date_converted = pd.to_datetime(parsed_date)

In [294]:
# Now the sorting phase:
sorting = parsed_date_converted.sort_values(ascending= True)
print(sorting)

# get the index as result of the function.
sorting.index

9     1971-04-10
84    1971-05-18
2     1971-07-08
53    1971-07-11
28    1971-09-12
         ...    
231   2016-05-01
141   2016-05-30
186   2016-10-13
161   2016-10-19
413   2016-11-01
Name: 0, Length: 500, dtype: datetime64[ns]


Int64Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
            ...
            152, 235, 464, 253, 427, 231, 141, 186, 161, 413],
           dtype='int64', length=500)

In [299]:
def date_sorter():
    s1 = '(\d{1,2}[\/|\-]\d{1,2}[\/|\-|\ ]\d{2,4})'
    s2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\-\s]\d{1,2}[,]{0,1}[\-\s]\d{2,4})'
    s3 = '(\d{1,2}[\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\-|\s]\d{2,4})'
    s4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\-|\s]\d{2,4}[\S]*[\s]\d{2,4})'
    s5 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[\-|\s]\d{2,4})'
    s6 = '(\d{1,2}[\/|\-][1|2]\d{3})'
    s7 = '([1|2]\d{3})'

    full_regex = '(%s|%s|%s|%s|%s|%s|%s)' %(s1, s2, s3, s4, s5, s6,s7)
    parsed_date = df.str.extract(full_regex)
    
    # cut the df to make a series from 1st column.
    parsed_date = parsed_date.iloc[:,0]

    # correct the spelling mistakes.
    parsed_date = parsed_date.str.replace('Janaury', 'January')
    parsed_date = parsed_date.str.replace('Decemeber', 'December')

    # finally, convert to datetime form.
    # That will apply all the rules were given above.
    parsed_date_converted = pd.to_datetime(parsed_date)
    
    # Now the sorting phase:
    sorting = parsed_date_converted.sort_values(ascending= True)
    # get the index as result of the function.
    # put the indexes into Pd Series.
    sorting_indexes = pd.Series(sorting.index)
        
    return sorting_indexes

date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64